# Testing EUGENE training

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing the training of EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

# Set-up

In [1]:
import numpy as np
import pandas as pd

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [2]:
from eugene.dataloading.SeqDataModule import SeqDataModule

# Benchmark params

In [4]:
BATCH_SIZE = 32
NUM_SEQS = 100
SEQ_LEN = 66
DATA_TYPE = "tsv"
NUM_WORKERS = 0

In [5]:
CNN_KWARGS=dict(channels=[4, 16, 32], 
                conv_kernels=[15, 5], 
                pool_kernels=[1, 1])
RNN_KWARGS=dict(output_dim=32,
                bidirectional=True,
                batch_first=True)
FCN_KWARGS=dict(hidden_dims=[50], 
                output_dim=1)

In [6]:
MODEL = "hybrid"
STRAND = "ts"
TASK = "binary_classification"

# Instantiate model

In [456]:
from eugene.models.EUGENE import EUGENE

In [462]:
EUGENE(model="FCN", 
       input_len=66,
       strand=STRAND,
       task=TASK,
       #conv_kwargs=CNN_KWARGS,
       #rnn_kwargs=RNN_KWARGS,
       fc_kwargs=FCN_KWARGS)

EUGENE(
  (model): FCN(
    (fcn): BasicFullyConnectedModule(
      (module): Sequential(
        (0): Linear(in_features=264, out_features=50, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=50, out_features=1, bias=True)
      )
    )
    (reverse_fcn): BasicFullyConnectedModule(
      (module): Sequential(
        (0): Linear(in_features=264, out_features=50, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=50, out_features=1, bias=True)
      )
    )
    (accuracy): Accuracy()
    (auroc): AUROC()
  )
)

In [422]:
import claim.utils as cu
from eugene.models.FCN import FCN
from eugene.models.CNN import CNN
from eugene.models.RNN import RNN
from eugene.models.hybrid import hybrid

In [423]:
eugene = hybrid(input_len=66,
                strand=STRAND,
                task=TASK,
                conv_kwargs=CNN_KWARGS,
                rnn_kwargs=RNN_KWARGS,
                fc_kwargs=FCN_KWARGS)
cu.init_weights(eugene)
eugene

hybrid(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (4): ReLU(inplace=True)
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (recurrentnet): BasicRecurrent(
    (module): LSTM(32, 32, batch_first=True, bidirectional=True)
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=64, out_features=50, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=50, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [424]:
next(eugene.parameters())[0][:10]

tensor([[ 7.7503e-02, -1.2629e-01, -4.3062e-02,  9.6357e-02,  1.8623e-02,
          9.9583e-02,  7.7533e-02,  2.3856e-02,  1.1252e-01,  1.1521e-01,
         -1.7128e-02, -1.1266e-01, -2.4774e-02,  6.4558e-02, -9.0405e-03],
        [-7.3737e-02, -3.4331e-03,  8.4321e-02, -1.2849e-01, -7.5818e-03,
          1.0489e-01, -4.8266e-02,  1.2605e-01, -1.1509e-01,  5.4411e-03,
          1.4926e-02,  4.1321e-02,  1.2232e-01,  1.1141e-01, -6.9318e-02],
        [ 8.1646e-02, -9.1688e-02,  1.1525e-01, -1.1561e-01,  2.0448e-03,
         -4.3314e-02,  7.5207e-02,  4.1196e-02, -1.0294e-01, -1.1241e-01,
         -1.0293e-04,  2.8233e-02,  7.0684e-02, -1.2404e-01,  2.9777e-02],
        [-1.7438e-02, -2.5851e-02, -2.0042e-02,  1.3051e-03, -3.3175e-02,
         -1.3970e-02, -6.7433e-02,  1.4657e-02, -1.2001e-02,  9.7930e-02,
          9.6330e-02, -6.0111e-02,  6.7024e-02, -3.1758e-02, -4.1575e-02]],
       grad_fn=<SliceBackward0>)

# Load data

In [425]:
from torchvision import transforms
from eugene.utils.seq_transforms import ReverseComplement, Augment, OneHotEncode, ToTensor

In [426]:
data_transform = transforms.Compose([
    Augment(randomize_linker_p=0.1, enhancer="WT-otx-a"), 
    ReverseComplement(ohe_encoded=False), 
    OneHotEncode(), 
    ToTensor(transpose=True),
    #ToTensor(transpose=False)
])

In [12]:
import numpy as np
DATA_DIR = "test_{}seqs_{}/".format(NUM_SEQS, SEQ_LEN)
np.load(DATA_DIR + "test_ohe_seqs.npy").shape

(100, 66, 4)

In [427]:
DATA_DIR = "test_{}seqs_{}/".format(NUM_SEQS, SEQ_LEN)
mod = SeqDataModule(seq_file=DATA_DIR + "test_seqs.tsv",
                    transform=data_transform,
                    num_workers=4,
                    batch_size=BATCH_SIZE,
                    #load_kwargs=dict(target_file=DATA_DIR + "test_labels.npy", rev_seq_file=DATA_DIR + "test_rev_ohe_seqs.npy"),
                    load_kwargs=dict(seq_col="SEQ", target_col="LABEL")
                   )

# Train (and time) with PyTorch Lightning

In [428]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [429]:
LOG_DIR = f"../test/test_logs/batch_size-{BATCH_SIZE}.num_workers-{NUM_WORKERS}.data_type-{DATA_TYPE}.num_seq-{NUM_SEQS}.seq_len-{SEQ_LEN}"
logger = TensorBoardLogger(LOG_DIR, name=MODEL, version=f"{STRAND}_{TASK}")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [430]:
trainer.fit(model=eugene, datamodule=mod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name         | Type                      | Params
-----------------------------------------------------------
0 | convnet      | BasicConv1D               | 3.6 K 
1 | recurrentnet | BasicRecurrent            | 16.9 K
2 | fcnet        | BasicFullyConnectedModule | 3.3 K 
3 | accuracy     | Accuracy                  | 0     
4 | auroc        | AUROC                     | 0     
-----------------------------------------------------------
23.8 K    Trainable params
0         Non-trainable params
23.8 K    Total params
0.095     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Scratch

In [434]:
import importlib

## Old Benchmark dsEUGENE
<div class="alert alert-info" role="alert">
  <b>Benchmark dsEUGENE models</b>
</div>

In [10]:
BATCH_SIZE = 32
DATA_TYPE = "tsv"
NUM_WORKERS = 0
NUM_SEQS = 100
SEQ_LEN = 66

### Load data

In [11]:
from torchvision import transforms
from eugene.utils.seq_transforms import ReverseComplement, Augment, OneHotEncode, ToTensor

In [12]:
data_transform = transforms.Compose([
    Augment(randomize_linker_p=0.1, enhancer="WT-otx-a"), 
    ReverseComplement(ohe_encoded=False), 
    OneHotEncode(), 
    ToTensor(transpose=True)
])

In [13]:
DATA_DIR = "test_{}seqs_{}/".format(NUM_SEQS, SEQ_LEN)
mod = SeqDataModule(seq_file=DATA_DIR + "test_seqs.tsv",
                    transform=data_transform,
                    num_workers=4,
                    batch_size=BATCH_SIZE,
                    load_kwargs=dict(seq_col="SEQ", target_col="ACTIVITY", low_thresh=0.09, high_thresh=0.4))
#load_kwargs=dict(target_file=DATA_DIR + "test_labels.npy", rev_seq_file=DATA_DIR + "test_rev_ohe_seqs.npy"))

### Instantiate EUGENE architecture: dsEUGENE

In [15]:
import claim.utils as cu
from eugene.models.dsEUGENE import dsEUGENE

In [16]:
cnn=dict(input_len=66, channels=[4, 16, 32], conv_kernels=[15, 5], pool_kernels=[1, 1])
rnn=dict(output_dim=32, batch_first=True)
fc=dict(output_dim=1)

In [17]:
eugene = dsEUGENE(conv_kwargs=cnn, rnn_kwargs=rnn, fc_kwargs=fc)
cu.init_weights(eugene)
eugene

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


dsEUGENE(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (4): ReLU(inplace=True)
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (recurrentnet): BasicRecurrent(
    (module): LSTM(64, 32, batch_first=True)
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=32, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [18]:
next(eugene.parameters())[0]

tensor([[ 0.0976,  0.0978,  0.0585,  0.1228, -0.0595,  0.1031,  0.0438,  0.1063,
          0.0036, -0.1268, -0.1015, -0.0787, -0.1273,  0.1059, -0.0299],
        [ 0.0327,  0.1275, -0.0565, -0.0433,  0.0588, -0.0395,  0.0510, -0.0235,
         -0.0305,  0.0379, -0.0716, -0.0139,  0.0997, -0.0119,  0.0234],
        [ 0.0125, -0.1040, -0.0366,  0.0776, -0.0395,  0.0229,  0.0047, -0.0082,
         -0.0459,  0.0842,  0.1201,  0.0976,  0.0154,  0.0340, -0.0016],
        [ 0.1215,  0.0317,  0.1269, -0.0114,  0.1286, -0.1020, -0.1156, -0.0081,
          0.0828,  0.0014,  0.0338, -0.0051,  0.0270,  0.1262, -0.0590]],
       grad_fn=<SelectBackward0>)

### Train (and time) with PyTorch Lightning

In [19]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [20]:
LOG_DIR = "../test/test_logs/batch_size-{0}.num_workers-{1}.data_type-{2}.num_seq-{3}.seq_len-{4}".format(BATCH_SIZE, NUM_WORKERS, DATA_TYPE, NUM_SEQS, SEQ_LEN)
logger = TensorBoardLogger(LOG_DIR, name="dsEUGENE")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [21]:
trainer.fit(model=eugene, datamodule=mod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name         | Type                      | Params
-----------------------------------------------------------
0 | convnet      | BasicConv1D               | 3.6 K 
1 | recurrentnet | BasicRecurrent            | 12.5 K
2 | fcnet        | BasicFullyConnectedModule | 33    
3 | accuracy     | Accuracy                  | 0     
4 | auroc        | AUROC                     | 0     
-----------------------------------------------------------
16.1 K    Trainable params
0         Non-trainable params
16.1 K    Total params
0.065     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [22]:
# Sanity check batch size
for i, batch in enumerate(mod.train_dataloader()):
    print(i, batch[0].size())
    if i == 3:
        break

0 torch.Size([32, 1])
1 torch.Size([32, 1])


In [23]:
del logger, trainer, mod, eugene

In [ ]:
%whos

## OLS Dataset

In [6]:
DATA_DIR = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/2021_OLS_Library.tsv"
mod = SeqDataModule(seq_file=DATA_DIR,
                    transform=data_transform,
                    num_workers=4,
                    batch_size=BATCH_SIZE,
                    load_kwargs=dict(seq_col="SEQ", target_col="ACTIVITY_SUMRNA_NUMDNA", low_thresh=0.09, high_thresh=0.4))
#load_kwargs=dict(target_file=DATA_DIR + "test_labels.npy", rev_seq_file=DATA_DIR + "test_rev_ohe_seqs.npy"))

In [20]:
# Check the DataLoader
for i_batch, sample_batched in enumerate(mod.train_dataloader()):
    _, x, x_rev_comp, y = sample_batched
    outs = cnn(x, x_rev_comp).squeeze(dim=1)
    print(outs.size())
    if i_batch == 3:
        break

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1554c0618950>
Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])


In [104]:
mod.setup()

In [105]:
tar = mod.train_dataloader().dataset

In [106]:
np.isinf(tar.dataset.targets).sum()

0

In [18]:
trainer = pl.Trainer(gpus=1, max_epochs=2)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [19]:
trainer.fit(model=cnn, datamodule=mod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name      | Type                      | Params
--------------------------------------------------------
0 | convnet   | BasicConv1D               | 3.6 K 
1 | fcnet     | BasicFullyConnectedModule | 1.5 K 
2 | r_squared | R2Score                   | 0     
--------------------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1554c0618950>
Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x1554c0618950>
Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/multiproce

In [11]:
import torch.nn.functional as F

In [12]:
F.mse_loss(outs, y)

NameError: name 'outs' is not defined

## Random dataset
<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of our eugene architectures on some random data</b>
</div>

### Load data

In [29]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader

### Training set
training_dataset = dataset.TensorDataset(torch.randn(1000, 4, 66), torch.bernoulli(torch.empty(1000).uniform_(0,1)))
training_dataloader = DataLoader(training_dataset, batch_size=32, num_workers=1)
training_dataset[0][0].shape, training_dataset[0][1]

for i_batch, batch in enumerate(training_dataloader):
    x, y = batch
    outs = eugene(x)
    print(x.shape, y.shape, outs.shape)
    if i_batch==3:
        break

### Validation set
validation_dataset = dataset.TensorDataset(torch.randn(100, 4, 66), torch.bernoulli(torch.empty(100).uniform_(0,1)))
validation_dataloader = DataLoader(validation_dataset, batch_size=32, num_workers=4)
validation_dataset[0][0].shape, validation_dataset[0][1]

for i_batch, batch in enumerate(validation_dataloader):
    x, y = batch
    outs = eugene(x)
    print(x.shape, y.shape, outs.shape)
    if i_batch==3:
        break

## Instantiate EUGENE architecture: ssEUGENE
cnn=dict(input_len=66, channels=[4, 16], conv_kernels=[15, 5], pool_kernels=[1, 1])
rnn=dict(output_dim=32, batch_first=True)
fc=dict(output_dim=1)

from ssEUGENE import ssEUGENE
eugene = ssEUGENE(conv_kwargs=cnn, rnn_kwargs=rnn, fc_kwargs=fc)
eugene

x = torch.randn(10, 4, 66)
out = eugene(x)
out.shape

### Training with PyTorch Lightning

In [36]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("random_test", name="ssEUGENE")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

trainer.fit(model=eugene, train_dataloader=training_dataloader, val_dataloaders=validation_dataloader)

## MPRADataset
<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of our eugene architectures on a MPRA dataset</b>
</div>

### Load data and instantiate EUGENE architecture: dsEUGENE

In [4]:
from eugene.load_data import load_csv, load_numpy
from eugene.MPRADataset import MPRADataset
from torch.utils.data import DataLoader
from torchvision import transforms
from transforms import ReverseComplement, Augment, OneHotEncode, ToTensor
from dsEUGENE import dsEUGENE
import claim.utils as cu

TRAIN_SEQ = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/seqs/0.09-0.4_seqs-train-0.9.txt"
TRAIN_LABEL = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/binary/0.09-0.4_y-train-0.9_binary.txt"
train_seqs, train_targets = load_numpy(TRAIN_SEQ, TRAIN_LABEL, is_seq_text=True)
len(train_seqs), train_seqs[0], len(train_targets), train_targets[0]

VAL_SEQ = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/seqs/0.09-0.4_seqs-test-0.1.txt"
VAL_LABEL = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/binary/0.09-0.4_y-test-0.1_binary.txt"
val_seqs, val_targets = load_numpy(VAL_SEQ, VAL_LABEL, is_seq_text=True)
len(val_seqs), val_seqs[0], len(val_targets), val_targets[0]

# Compose different data transforms for this particular load
data_transform = transforms.Compose([
    Augment(randomize_linker_p=0.1, enhancer="WT-otx-a"), ReverseComplement(ohe_encoded=False), OneHotEncode(), ToTensor(transpose=True)
])

# Instantiate a Dataset
train_dataset = MPRADataset(train_seqs, train_targets, transform=data_transform)
val_dataset = MPRADataset(val_seqs, val_targets, transform=data_transform)

# Instantiate a DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=4)

for i_batch, batch in enumerate(train_dataloader):
    x, x_rev_comp, y = batch["sequence"], batch["reverse_complement"], batch["target"]
    outs = eugene(x, x_rev_comp)
    print(x.shape, x_rev_comp.shape, y.shape, outs.shape)
    if i_batch==3:
        break
        
for i_batch, batch in enumerate(val_dataloader):
    x, x_rev_comp, y = batch["sequence"], batch["reverse_complement"], batch["target"]
    outs = eugene(x, x_rev_comp)
    print(x.shape, x_rev_comp.shape, y.shape, outs.shape)
    if i_batch==3:
        break
        
cnn=dict(input_len=66, channels=[4, 16], conv_kernels=[15, 5], pool_kernels=[1, 1])
rnn=dict(output_dim=32, batch_first=True)
fc=dict(output_dim=1)

eugene = dsEUGENE(conv_kwargs=cnn, rnn_kwargs=rnn, fc_kwargs=fc)
cu.init_weights(eugene)

torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])
torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])
torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])
torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])
torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])
torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])
torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])
torch.Size([512, 4, 66]) torch.Size([512, 4, 66]) torch.Size([512]) torch.Size([512, 1])


### Training with PyTorch Lightning

In [7]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("0.18-0.4_test", name="dsEUGENE")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

trainer.fit(model=eugene, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# References